In [1]:
import utilities
import pandas as pd

df = pd.read_csv('./R/data/dow_financials.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [2]:
df = df.dropna()

conditions = (df['Revenue_Growth'] > 0) & \
            (df['GPMargin'] > 0)& \
            (df['EPS_Growth'] > 0)& \
            (df['ROE'] > 0) & \
            (df['DPS_Growth'] > 0) & \
            (df['DivYield'] > 0)
df = df[conditions]

In [3]:
# Compute historical momentum
# Average momentum of prev 6M, 12M and 24M

mom_list = []
for symbol in df['symbol']:
    print(symbol)
    m6_momentum = utilities.calculate_hist_momentum(symbol, 120)
    m12_momentum = utilities.calculate_hist_momentum(symbol, 252)
    m24_momentum = utilities.calculate_hist_momentum(symbol, 504)
    avg_momentum = (m6_momentum + m12_momentum + m24_momentum) / 3
    mom_list.append(avg_momentum)

df['momentum'] = mom_list
df

WBA
NKE
UNH
TRV
VZ
INTC
JNJ
JPM
AMGN
GS
CSCO
MSFT
HD


,symbol,name,exchange,sector,industry,marketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,momentum
1,WBA,"Walgreens Boots Alliance, Inc.",NASDAQ,Healthcare,Pharmaceutical Retailers,40.507343,0.441947,0.106518,0.202202,0.163854,0.039964,1.8715,0.304819,0.059511
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,266.928734,0.596394,0.514070,0.448202,0.039428,0.006341,1.0700,0.717702,0.679403
4,UNH,UnitedHealth Group Incorporated,NYSE,Healthcare,Healthcare Plans,392.174240,0.587774,0.258991,0.257903,1.204764,0.012513,5.2000,1.499471,0.392003
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,37.161980,0.481540,0.132920,0.249376,0.280277,0.023028,3.4300,0.317382,0.142344
6,VZ,Verizon Communications Inc.,NYSE,Communication Services,Telecom Services,230.853083,0.089299,0.283399,0.586553,0.102362,0.045032,2.5110,0.110776,-0.026397
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,217.779765,0.743302,0.231499,0.555936,0.506024,0.031716,1.7025,0.374296,0.037014
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,453.497356,0.360444,0.235654,0.662693,2.559538,0.023742,4.0900,0.369355,0.207709
12,JPM,JPMorgan Chase & Co.,NYSE,Financial Services,Banks—Diversified,451.726180,0.657863,0.148099,0.236408,0.186842,0.023814,3.6000,0.959192,0.375574
15,AMGN,Amgen Inc.,NASDAQ,Healthcare,Drug Manufacturers—General,137.772302,0.393334,0.702163,0.756160,0.032353,0.035364,8.4800,0.763173,0.036395
18,GS,"The Goldman Sachs Group, Inc.",NYSE,Financial Services,Capital Markets,128.494584,2.875376,0.160330,0.267396,0.537204,0.013221,5.0000,0.759952,0.770487


In [4]:
df['mom_rank'] = df['momentum'].rank()
df = df.sort_values(by=['mom_rank'], ascending=False)
numRows = df.shape[0]
numCols = df.shape[1]
top10 = df[:10].copy()

In [5]:
top10['DPSGrowth_Rank'] = top10['DPS_Growth'].rank()

In [6]:
top10 = top10.sort_values(by='DPSGrowth_Rank', ascending=False)
top10

,symbol,name,exchange,sector,industry,marketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,momentum,mom_rank,DPSGrowth_Rank
4,UNH,UnitedHealth Group Incorporated,NYSE,Healthcare,Healthcare Plans,392.174240,0.587774,0.258991,0.257903,1.204764,0.012513,5.2000,1.499471,0.392003,10.0,10.0
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,347.919942,0.925909,9.575875,0.339370,0.453451,0.019253,6.3000,1.407165,0.365775,8.0,9.0
12,JPM,JPMorgan Chase & Co.,NYSE,Financial Services,Banks—Diversified,451.726180,0.657863,0.148099,0.236408,0.186842,0.023814,3.6000,0.959192,0.375574,9.0,8.0
18,GS,"The Goldman Sachs Group, Inc.",NYSE,Financial Services,Capital Markets,128.494584,2.875376,0.160330,0.267396,0.537204,0.013221,5.0000,0.759952,0.770487,13.0,7.0
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,266.928734,0.596394,0.514070,0.448202,0.039428,0.006341,1.0700,0.717702,0.679403,12.0,6.0
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2140.391080,1.332562,0.462317,0.689258,0.068293,0.009655,2.7500,0.556311,0.613702,11.0,5.0
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,233.677390,0.272511,0.263007,0.639438,0.133333,0.026330,1.4600,0.422486,0.261422,7.0,4.0
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,453.497356,0.360444,0.235654,0.662693,2.559538,0.023742,4.0900,0.369355,0.207709,6.0,3.0
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,37.161980,0.481540,0.132920,0.249376,0.280277,0.023028,3.4300,0.317382,0.142344,5.0,2.0
1,WBA,"Walgreens Boots Alliance, Inc.",NASDAQ,Healthcare,Pharmaceutical Retailers,40.507343,0.441947,0.106518,0.202202,0.163854,0.039964,1.8715,0.304819,0.059511,4.0,1.0


In [7]:
watchlist = top10[['symbol','name']]
watchlist

,symbol,name
4,UNH,UnitedHealth Group Incorporated
21,HD,"The Home Depot, Inc."
12,JPM,JPMorgan Chase & Co.
18,GS,"The Goldman Sachs Group, Inc."
3,NKE,"NIKE, Inc."
20,MSFT,Microsoft Corporation
19,CSCO,"Cisco Systems, Inc."
9,JNJ,Johnson & Johnson
5,TRV,"The Travelers Companies, Inc."
1,WBA,"Walgreens Boots Alliance, Inc."


In [11]:
# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []

for symbol in watchlist['symbol']:
    currentPrice = utilities.get_current_price(symbol)
    high = utilities.calculate_prev_max_high(symbol, 252)
    discount_pct = (high - currentPrice) / high
    
    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)
    

watchlist['CurrentPrice'] = currentPrices
watchlist['52W_High'] = highs
watchlist['Discount%'] = discounts
    

<ipython-input-11-0e97d29e27fd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watchlist['CurrentPrice'] = currentPrices
<ipython-input-11-0e97d29e27fd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watchlist['52W_High'] = highs
<ipython-input-11-0e97d29e27fd>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [12]:
watchlist

,symbol,name,CurrentPrice,52W_High,Discount%
4,UNH,UnitedHealth Group Incorporated,409.67,425.980011,0.038288
21,HD,"The Home Depot, Inc.",331.32,345.690002,0.041569
12,JPM,JPMorgan Chase & Co.,159.98,167.440002,0.044553
18,GS,"The Goldman Sachs Group, Inc.",410.78,418.620000,0.018728
3,NKE,"NIKE, Inc.",171.69,174.380000,0.015426
20,MSFT,Microsoft Corporation,292.85,291.540000,-0.004493
19,CSCO,"Cisco Systems, Inc.",56.47,56.450000,-0.000354
9,JNJ,Johnson & Johnson,176.25,175.280000,-0.005534
5,TRV,"The Travelers Companies, Inc.",156.31,162.710007,0.039334
1,WBA,"Walgreens Boots Alliance, Inc.",49.29,57.049999,0.136021
